# GoogLeNet (Inception v1) の実装

このノートブックでは、2014年のImageNet LSVRCで優勝したGoogLeNet（別名 Inception v1）のアーキテクチャについて学び、その中核となる**Inceptionモジュール**と**1x1畳み込みによる次元削減**のアイデアを理解します。
NumPyで主要な概念を実装・確認した後、PyTorchを使ってCIFAR-10データセット用に調整したGoogLeNet風モデルを実装し、学習と評価を行います。

**参考論文:**
*   Szegedy, C., Liu, W., Jia, Y., Sermanet, P., Reed, S., Anguelov, D., ... & Rabinovich, A. (2015). Going deeper with convolutions. In *Proceedings of the IEEE conference on computer vision and pattern recognition* (pp. 1-9).

**このノートブックで学ぶこと:**
1.  GoogLeNet (Inception) の設計思想とアーキテクチャの概要
2.  主要なコンポーネントの理解とNumPyによる概念実装:
    *   Inceptionモジュールの構造
    *   1x1畳み込みによる次元削減（ボトルネック層）
    *   Global Average Pooling (GAP)
3.  PyTorchを使ったGoogLeNet風モデル（CIFAR-10用）の実装
4.  補助分類器 (Auxiliary Classifiers) の概念と実装
5.  CIFAR-10データセットでの学習と評価

**前提知識:**
*   VGGのノートブックで学んだCNNの基礎とPyTorchによる実装経験

## 1. 必要なライブラリのインポート


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F # 活性化関数やプーリングなどで使用
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

print(f"PyTorch Version: {torch.__version__}")
print(f"Torchvision Version: {torchvision.__version__}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

PyTorch Version: 2.5.0+cu124
Torchvision Version: 0.20.0+cu124
Using device: cuda


## 2. GoogLeNetの主要な設計思想とNumPyによる概念実装

GoogLeNetの核心は、計算資源を効率的に利用しながらネットワークの深さと幅を増やすための**Inceptionモジュール**です。

### 2.1 Inceptionモジュール と 1x1畳み込みによる次元削減

*   **Inceptionモジュールの概念 (論文 Figure 2):**
    ![Inception](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-22_at_3.22.39_PM.png)  
    従来のCNNでは、各層でどのサイズのカーネル（例: 1x1, 3x3, 5x5）やプーリングを使うかを事前に一つ選択する必要がありました。Inceptionモジュールは、これらの異なる演算を**並列**に行い、その結果をチャネル方向に連結（concatenate）することで、ネットワーク自身に最適な特徴の組み合わせを学習させようとします。

    **ナイーブなInceptionモジュール (Figure 2a):**
    1.  1x1畳み込み
    2.  3x3畳み込み
    3.  5x5畳み込み
    4.  3x3マックスプーリング
    これらの出力を全てチャネル方向に連結します。

    **問題点:** 3x3や特に5x5の畳み込みは、入力チャネル数が多い場合、計算コストが非常に高くなります。また、プーリング層も出力チャネル数を維持するため、連結後のチャネル数がどんどん増大してしまいます。

*   **次元削減を伴うInceptionモジュール (Figure 2b - GoogLeNetで採用):**
    この問題を解決するため、計算コストの高い3x3畳み込みと5x5畳み込みの**前**、およびマックスプーリングの**後**に**1x1畳み込み（ボトルネック層）**を挿入し、入力チャネル数を削減します。

    **1x1畳み込みの役割:**
    1.  **次元削減:** 出力チャネル数を入力チャネル数より小さく設定することで、後続の畳み込み演算の計算量を大幅に削減できます。
    2.  **特徴の線形結合と非線形性の追加:** チャネル間で情報を混ぜ合わせ、ReLUを適用することで非線形性を加えます。これはNetwork in Network [Lin et al., 2013] のアイデアに基づいています。

    **次元削減版Inceptionモジュールの構成例:**
    1.  ブランチ1: 1x1畳み込み
    2.  ブランチ2: 1x1畳み込み（次元削減） → 3x3畳み込み
    3.  ブランチ3: 1x1畳み込み（次元削減） → 5x5畳み込み
    4.  ブランチ4: 3x3マックスプーリング → 1x1畳み込み（次元削減/特徴変換）
    これらの4つのブランチの出力をチャネル方向に連結します。

#### 2.1.1 NumPyによる1x1畳み込み (次元削減) の概念確認
(VGGノートブックの `convolve1x1` を再利用)
ここでは、入力チャネル数を減らす例を示します。

In [5]:
def convolve1x1_simple(input_volume, kernels_out_ch_in_ch, biases_out_ch):
    """
    簡易版1x1畳み込み (入力: C_in x H x W, カーネル: C_out x C_in)
    出力: C_out x H x W
    """
    C_in, H, W = input_volume.shape
    C_out, _ = kernels_out_ch_in_ch.shape # (C_out, C_in)
    
    output_volume = np.zeros((C_out, H, W))
    for h_idx in range(H):
        for w_idx in range(W):
            pixel_vector = input_volume[:, h_idx, w_idx] # (C_in,)
            # (C_out, C_in) @ (C_in,) -> (C_out,)
            transformed_pixel = np.dot(kernels_out_ch_in_ch, pixel_vector) + biases_out_ch
            output_volume[:, h_idx, w_idx] = transformed_pixel
    return output_volume

In [8]:
print("--- 1x1 畳み込み (次元削減) テスト ---")
# 4チャネル入力画像 (2x2)
test_input_bottleneck = np.random.rand(4, 2, 2).astype(np.float32)
print("入力 (4チャネル x 2x2):\n", test_input_bottleneck.shape)

# 2出力チャネルに削減する1x1カーネル (2出力 x 4入力)
kernels_bottleneck = np.random.rand(2, 4).astype(np.float32) * 0.1
biases_bottleneck = np.random.rand(2).astype(np.float32) * 0.01

output_bottleneck = convolve1x1_simple(test_input_bottleneck, kernels_bottleneck, biases_bottleneck)
print("\n1x1畳み込み (次元削減) 後 (2チャネル x 2x2):\n", output_bottleneck.shape)
# print(output_bottleneck)

--- 1x1 畳み込み (次元削減) テスト ---
入力 (4チャネル x 2x2):
 (4, 2, 2)

1x1畳み込み (次元削減) 後 (2チャネル x 2x2):
 (2, 2, 2)


### 2.2 Global Average Pooling (GAP)

*   **概念:**
    従来のCNNでは、畳み込み層/プーリング層の後にいくつかの全結合層を配置して最終的な分類を行っていました。全結合層は多くのパラメータを持ち、過学習の原因となりやすいです。
    GoogLeNet (や他の現代的なCNN) では、最後の畳み込み層（またはInceptionモジュール）の出力特徴マップに対して、**Global Average Pooling (GAP)** を適用します。
    GAPは、各特徴マップの空間的な次元（高さと幅）に対して平均値を計算し、チャネルごとに1つの値を出力します。
    例えば、入力が `(C, H, W)` の特徴マップであれば、GAPの出力は `(C, 1, 1)` となり、これをフラット化して `(C)` 次元のベクトルとします。
    このベクトルを直接Softmax層に入力するか、オプションで1つだけ全結合層を挟むこともあります。

*   **利点:**
    *   **パラメータ数の大幅な削減:** 全結合層の代わりに使うことで、モデル全体のパラメータ数を大きく減らせます。
    *   **過学習の抑制:** パラメータが少ないため、正則化効果があります。
    *   **空間情報への頑健性:** 入力画像の空間的な位置ずれに対して、より頑健になります。
    *   **入力画像サイズへの柔軟性:** 理論上は、GAPは任意の入力サイズの特徴マップに適用可能です（ただし、学習時と推論時で分布が変わる可能性には注意）。

*   **NumPyによる概念実装:**

In [10]:
def global_average_pool(feature_maps):
    """
    Global Average Pooling (NumPy実装)
    Args:
        feature_maps (np.array): 入力特徴マップ (チャネル数, 高さ, 幅)
                                  または (バッチサイズ, チャネル数, 高さ, 幅)
    Returns:
        np.array: プーリング後のベクトル (チャネル数,) または (バッチサイズ, チャネル数)
    """
    if feature_maps.ndim == 3: # (C, H, W)
        # HとWの次元 (axis 1と2) に沿って平均を取る
        return np.mean(feature_maps, axis=(1, 2))
    elif feature_maps.ndim == 4: # (B, C, H, W)
        # HとWの次元 (axis 2と3) に沿って平均を取る
        return np.mean(feature_maps, axis=(2, 3))
    else:
        raise ValueError("Input must be 3D or 4D tensor.")

In [11]:
print("\n--- Global Average Pooling テスト ---")
# 2チャネル、3x3の特徴マップ
test_gap_input = np.array([
    [[1,2,3], [4,5,6], [7,8,9]], # Channel 0
    [[9,8,7], [6,5,4], [3,2,1]]  # Channel 1
], dtype=np.float32)
print("GAP入力 (2チャネル x 3x3):\n", test_gap_input)
output_gap = global_average_pool(test_gap_input)
print("\nGAP出力 (2チャネル,):\n", output_gap)
# Ch0_mean = (1+2+3+4+5+6+7+8+9)/9 = 45/9 = 5
# Ch1_mean = (9+8+7+6+5+4+3+2+1)/9 = 45/9 = 5
# 期待値: [5., 5.]

# バッチ処理の場合
test_gap_input_batch = np.random.rand(2, 3, 4, 4).astype(np.float32) # (batch=2, ch=3, H=4, W=4)
print("\nGAP入力 (バッチ, 2x3x4x4):\n", test_gap_input_batch.shape)
output_gap_batch = global_average_pool(test_gap_input_batch)
print("\nGAP出力 (バッチ, 2x3):\n", output_gap_batch.shape)


--- Global Average Pooling テスト ---
GAP入力 (2チャネル x 3x3):
 [[[1. 2. 3.]
  [4. 5. 6.]
  [7. 8. 9.]]

 [[9. 8. 7.]
  [6. 5. 4.]
  [3. 2. 1.]]]

GAP出力 (2チャネル,):
 [5. 5.]

GAP入力 (バッチ, 2x3x4x4):
 (2, 3, 4, 4)

GAP出力 (バッチ, 2x3):
 (2, 3)


## 3. GoogLeNet アーキテクチャの概要

GoogLeNetは、複数のInceptionモジュールを積み重ね、途中と最後に補助分類器やGlobal Average Poolingを配置した構造です。
論文のTable 1に詳細なアーキテクチャが記載されています。
非常に深いネットワーク（22層の重み層）ですが、Inceptionモジュール内の1x1畳み込みによる次元削減のおかげで、パラメータ数はAlexNetよりも大幅に少なくなっています（約500万パラメータ vs AlexNetの約6000万）。

**主な構成要素:**
1.  初期の畳み込み層とプーリング層（ステム部分）
2.  複数のInceptionモジュールのスタック
3.  途中に配置される補助分類器（学習時のみ使用）
4.  Global Average Pooling層
5.  最終的な分類のための全結合層（または直接Softmax）とDropout

CIFAR-10用に実装する場合、入力画像サイズが小さいため、最初の畳み込み層のストライドやプーリング層の適用回数、Inceptionモジュール内のチャネル数などを大幅に調整する必要があります。

## 4. PyTorchによるGoogLeNet風モデルの実装とCIFAR-10での学習・評価

CIFAR-10データセット用に調整したGoogLeNet風モデル（Inceptionモジュールを含む）を実装します。
補助分類器も実装し、学習時に利用します

In [12]:
# CIFAR-10 データセットの準備 (VGGノートブックと同様)
transform_cifar_googlenet = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset_cifar_gn = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                      download=True, transform=transform_cifar_googlenet)
test_dataset_cifar_gn = torchvision.datasets.CIFAR10(root='./data', train=False,
                                                     download=True, transform=transform_cifar_googlenet)

batch_size_cifar_gn = 128 
train_loader_cifar_gn = DataLoader(train_dataset_cifar_gn, batch_size=batch_size_cifar_gn, shuffle=True, num_workers=2)
test_loader_cifar_gn = DataLoader(test_dataset_cifar_gn, batch_size=batch_size_cifar_gn, shuffle=False, num_workers=2)

classes_cifar_gn = ('plane', 'car', 'bird', 'cat', 'deer', 
                    'dog', 'frog', 'horse', 'ship', 'truck')

print(f"CIFAR-10 訓練データ数 (GoogLeNet): {len(train_dataset_cifar_gn)}")

Files already downloaded and verified
Files already downloaded and verified
CIFAR-10 訓練データ数 (GoogLeNet): 50000


In [13]:
# Inceptionモジュールの定義
class InceptionModule(nn.Module):
    def __init__(self, in_channels, n1x1, n3x3_reduce, n3x3, n5x5_reduce, n5x5, pool_proj):
        super(InceptionModule, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_channels, n1x1, kernel_size=1),
            nn.ReLU(True),
        )
        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_channels, n3x3_reduce, kernel_size=1),
            nn.ReLU(True),
            nn.Conv2d(n3x3_reduce, n3x3, kernel_size=3, padding=1),
            nn.ReLU(True),
        )
        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_channels, n5x5_reduce, kernel_size=1),
            nn.ReLU(True),
            nn.Conv2d(n5x5_reduce, n5x5, kernel_size=5, padding=2), # 5x5なのでpadding=2
            nn.ReLU(True),
        )
        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1), # 出力サイズを維持
            nn.Conv2d(in_channels, pool_proj, kernel_size=1),
            nn.ReLU(True),
        )

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        # 全てのブランチの出力をチャネル方向に連結
        return torch.cat([y1,y2,y3,y4], dim=1)

In [14]:
# 補助分類器の定義
class AuxiliaryClassifier(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(AuxiliaryClassifier, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((4, 4)) #論文では5x5だがCIFAR-10用に調整
        self.conv = nn.Conv2d(in_channels, 128, kernel_size=1) # 論文に倣う
        self.relu = nn.ReLU(True)
        self.fc1 = nn.Linear(128 * 4 * 4, 256) # 論文では1024だがCIFAR用に縮小
        self.dropout = nn.Dropout(0.7) # 論文では0.7
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.avgpool(x)
        x = self.conv(x)
        x = self.relu(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [15]:
# GoogLeNetモデル (CIFAR-10用に大幅に簡略化・調整)
class GoogLeNetCIFAR(nn.Module):
    def __init__(self, num_classes=10, aux_logits=True): # 訓練時に補助分類器を使うか
        super(GoogLeNetCIFAR, self).__init__()
        self.aux_logits = aux_logits

        # Stem (初期の畳み込み・プーリング層) - CIFAR-10用に調整
        self.pre_layers = nn.Sequential(
            # Input: 3x32x32
            nn.Conv2d(3, 64, kernel_size=3, padding=1), # 64x32x32
            nn.ReLU(True),
            # nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # -> 64x16x16 (論文風)
            # CIFAR-10では画像が小さいのでプーリングを減らすかカーネルを小さくする
        )
        
        # Inceptionモジュールのスタック
        # チャネル数はCIFAR-10用に大幅に削減
        self.inception3a = InceptionModule(64,  32, 32, 64,  8, 16, 16) # out: 32+64+16+16 = 128
        self.inception3b = InceptionModule(128, 64, 64, 96, 16, 48, 32) # out: 64+96+48+32 = 240
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # -> 240 x 16x16
        
        self.inception4a = InceptionModule(240, 96,  48, 104,  8, 24,  32) # out: 96+104+24+32=256
        if self.aux_logits: # 最初の補助分類器
            self.aux1 = AuxiliaryClassifier(256, num_classes) # 入力チャネル数変更
            
        self.inception4b = InceptionModule(256, 80,  56, 112, 12, 32,  32) # out: 80+112+32+32=256
        self.inception4c = InceptionModule(256, 64,  64, 128, 12, 32,  32) # out: 64+128+32+32=256
        self.inception4d = InceptionModule(256, 56,  72, 144, 16, 32,  32) # out: 56+144+32+32=264
        if self.aux_logits: # 2番目の補助分類器
            self.aux2 = AuxiliaryClassifier(264, num_classes) # 入力チャネル数変更

        self.inception4e = InceptionModule(264, 128, 80, 160, 16, 64, 64) # out: 128+160+64+64=416
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # -> 416 x 8x8
        
        self.inception5a = InceptionModule(416, 128, 80, 160, 16, 64, 64) # out: 128+160+64+64=416
        self.inception5b = InceptionModule(416, 192, 96, 192, 24, 64, 64) # out: 192+192+64+64=512

        # Global Average Pooling と最終分類器
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # 出力を1x1にする
        self.dropout = nn.Dropout(0.4) # 論文では0.4
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.pre_layers(x)
        
        out = self.inception3a(out)
        out = self.inception3b(out)
        out = self.maxpool3(out)
        
        out = self.inception4a(out)
        if self.training and self.aux_logits and hasattr(self, 'aux1'): # 訓練時のみ補助分類器を使用
            out_aux1 = self.aux1(out)
        else:
            out_aux1 = None
            
        out = self.inception4b(out)
        out = self.inception4c(out)
        out = self.inception4d(out)
        if self.training and self.aux_logits and hasattr(self, 'aux2'):
            out_aux2 = self.aux2(out)
        else:
            out_aux2 = None
            
        out = self.inception4e(out)
        out = self.maxpool4(out)
        
        out = self.inception5a(out)
        out = self.inception5b(out)
        
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.dropout(out)
        out_final = self.fc(out)
        
        if self.training and self.aux_logits:
            return out_final, out_aux1, out_aux2
        return out_final


In [16]:
model_googlenet_cifar = GoogLeNetCIFAR(num_classes=10, aux_logits=True).to(device)
print("\nGoogLeNet-style Model for CIFAR-10:\n")
# print(model_googlenet_cifar) # 非常に長いためコメントアウト

# ダミー入力でフォワードパスのテスト
dummy_input_gn = torch.randn(batch_size_cifar_gn // 4, 3, 32, 32).to(device) # バッチを小さく
try:
    # 訓練モードでの出力を確認
    model_googlenet_cifar.train()
    outputs_dummy_gn = model_googlenet_cifar(dummy_input_gn)
    if isinstance(outputs_dummy_gn, tuple): # 補助分類器あり
        print("Dummy Output Shapes (final, aux1, aux2):", 
              outputs_dummy_gn[0].shape, outputs_dummy_gn[1].shape, outputs_dummy_gn[2].shape)
    else: # 補助分類器なし (推論時など)
        print("Dummy Output Shape (final):", outputs_dummy_gn.shape)
except Exception as e:
    print("Error during GoogLeNet dummy forward pass:", e)


GoogLeNet-style Model for CIFAR-10:

Dummy Output Shapes (final, aux1, aux2): torch.Size([32, 10]) torch.Size([32, 10]) torch.Size([32, 10])


In [18]:
# 損失関数とOptimizer
criterion_gn = nn.CrossEntropyLoss()
# optimizer_gn = optim.SGD(model_googlenet_cifar.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
optimizer_gn = optim.Adam(model_googlenet_cifar.parameters(), lr=0.001)

# 学習率スケジューラ (オプション)
scheduler_gn = torch.optim.lr_scheduler.StepLR(optimizer_gn, step_size=10, gamma=0.1) # 論文では8エポック毎に4%減

# 学習ループ
num_epochs_gn = 40 # GoogLeNetは深いので学習に時間がかかる
print(f"\nGoogLeNet-style 学習開始 (CIFAR-10データ、{num_epochs_gn} epochs)...")

history_gn = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}

for epoch in range(num_epochs_gn):
    model_googlenet_cifar.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0
    start_epoch_time = time.time()
    
    for i, (images, labels) in enumerate(train_loader_cifar_gn):
        images, labels = images.to(device), labels.to(device)
        optimizer_gn.zero_grad()
        
        if model_googlenet_cifar.aux_logits: # 訓練時で補助分類器ありの場合
            output_final, output_aux1, output_aux2 = model_googlenet_cifar(images)
            loss_final = criterion_gn(output_final, labels)
            loss_aux1 = criterion_gn(output_aux1, labels)
            loss_aux2 = criterion_gn(output_aux2, labels)
            # 論文では補助分類器の損失に重み0.3をかける
            loss = loss_final + 0.3 * loss_aux1 + 0.3 * loss_aux2
        else:
            output_final = model_googlenet_cifar(images)
            loss = criterion_gn(output_final, labels)
            
        loss.backward()
        optimizer_gn.step()
        
        running_train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(output_final.data, 1) # 最終出力で精度計算
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_train_loss = running_train_loss / total_train
    epoch_train_acc = 100 * correct_train / total_train
    history_gn['train_loss'].append(epoch_train_loss)
    history_gn['train_acc'].append(epoch_train_acc)
    
    model_googlenet_cifar.eval() # 評価モード (補助分類器は使わない)
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images_test, labels_test in test_loader_cifar_gn:
            images_test, labels_test = images_test.to(device), labels_test.to(device)
            outputs_test_final = model_googlenet_cifar(images_test) # 推論時は最終出力のみ
            loss_test = criterion_gn(outputs_test_final, labels_test)
            running_test_loss += loss_test.item() * images_test.size(0)
            _, predicted_test = torch.max(outputs_test_final.data, 1)
            total_test += labels_test.size(0)
            correct_test += (predicted_test == labels_test).sum().item()
            
    epoch_test_loss = running_test_loss / total_test
    epoch_test_acc = 100 * correct_test / total_test
    history_gn['test_loss'].append(epoch_test_loss)
    history_gn['test_acc'].append(epoch_test_acc)
    
    end_epoch_time = time.time()
    epoch_duration = end_epoch_time - start_epoch_time
    
    print(f"Epoch [{epoch+1}/{num_epochs_gn}] - Dur: {epoch_duration:.1f}s - "
          f"TrL: {epoch_train_loss:.3f}, TrAcc: {epoch_train_acc:.2f}% - "
          f"TeL: {epoch_test_loss:.3f}, TeAcc: {epoch_test_acc:.2f}%")
    
    if scheduler_gn: scheduler_gn.step()

print("GoogLeNet-style 学習完了!")


GoogLeNet-style 学習開始 (CIFAR-10データ、40 epochs)...
Epoch [1/40] - Dur: 61.0s - TrL: 3.002, TrAcc: 26.87% - TeL: 1.702, TeAcc: 35.68%
Epoch [2/40] - Dur: 60.7s - TrL: 2.437, TrAcc: 42.13% - TeL: 1.426, TeAcc: 46.08%
Epoch [3/40] - Dur: 56.9s - TrL: 2.048, TrAcc: 53.18% - TeL: 1.182, TeAcc: 56.98%
Epoch [4/40] - Dur: 57.0s - TrL: 1.797, TrAcc: 59.94% - TeL: 1.051, TeAcc: 61.88%
Epoch [5/40] - Dur: 57.2s - TrL: 1.617, TrAcc: 64.96% - TeL: 0.908, TeAcc: 67.10%
Epoch [6/40] - Dur: 57.1s - TrL: 1.464, TrAcc: 68.93% - TeL: 0.854, TeAcc: 69.67%
Epoch [7/40] - Dur: 57.3s - TrL: 1.342, TrAcc: 71.67% - TeL: 0.806, TeAcc: 71.09%
Epoch [8/40] - Dur: 56.6s - TrL: 1.219, TrAcc: 74.44% - TeL: 0.751, TeAcc: 73.78%
Epoch [9/40] - Dur: 57.3s - TrL: 1.133, TrAcc: 76.71% - TeL: 0.725, TeAcc: 74.44%
Epoch [10/40] - Dur: 56.9s - TrL: 1.049, TrAcc: 78.41% - TeL: 0.702, TeAcc: 75.52%
Epoch [11/40] - Dur: 57.3s - TrL: 0.818, TrAcc: 84.17% - TeL: 0.628, TeAcc: 78.89%
Epoch [12/40] - Dur: 57.5s - TrL: 0.767, TrAcc:

KeyboardInterrupt: 

In [ ]:
# 学習曲線
plt.figure(figsize=(7, 3))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs_gn + 1), history_gn['train_loss'], label='Training Loss', marker='.')
plt.plot(range(1, num_epochs_gn + 1), history_gn['test_loss'], label='Test Loss', marker='.')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.title('GoogLeNet Training & Test Loss (CIFAR-10)'); plt.legend(); plt.grid(True)
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs_gn + 1), history_gn['train_acc'], label='Training Accuracy', marker='.')
plt.plot(range(1, num_epochs_gn + 1), history_gn['test_acc'], label='Test Accuracy', marker='.')
plt.xlabel('Epochs'); plt.ylabel('Accuracy (%)'); plt.title('GoogLeNet Training & Test Accuracy (CIFAR-10)'); plt.legend(); plt.ylim(0,100); plt.grid(True)
plt.tight_layout(); plt.show()

print(f"\n最終テスト精度 (GoogLeNet-style on CIFAR-10): {history_gn['test_acc'][-1]:.2f}%")

## 5. 考察

*   **GoogLeNet (Inception) の設計思想のポイント:**
    *   **Inceptionモジュール:** 異なるサイズの畳み込みとプーリングを並列に適用し、結果を連結することで、ネットワークが様々なスケールの特徴を効率的に学習できるようにしました。
    *   **1x1畳み込みによる次元削減:** Inceptionモジュール内の計算コストが高い畳み込みの前に1x1畳み込みを配置し、チャネル数を削減することで、計算量を大幅に抑えつつ深いネットワークを実現しました。これは「ボトルネック層」として機能します。
    *   **ネットワークの深さと幅の効率的な増加:** Inceptionモジュールを積み重ねることで、計算コストを適切に管理しながらネットワークを深く、かつ各層の「幅」（多様な演算の並列実行）も広げることができました。
    *   **補助分類器:** ネットワークの中間層に補助的な分類器を設け、学習時にその損失も考慮することで、勾配がネットワーク全体に行き渡りやすくなり、深いネットワークの学習を助ける効果がありました。また、正則化の効果も期待されます。
    *   **Global Average Pooling (GAP):** 従来の全結合層の代わりにGAPを使用することで、パラメータ数を大幅に削減し、過学習を抑制しました。

*   **PyTorchによるGoogLeNet風モデルの実装 (CIFAR-10用):**
    *   Inceptionモジュールと補助分類器をそれぞれ`nn.Module`として定義し、これらを組み合わせて全体のGoogLeNet風アーキテクチャを構築しました。
    *   CIFAR-10の画像サイズ (32x32) に合わせて、元のGoogLeNet (ImageNet用、入力224x224) からプーリング層の数やInceptionモジュール内のチャネル数を大幅に調整・削減しました。そうしないと、特徴マップの空間次元が早々に小さくなりすぎるか、パラメータ数が過大になります。
    *   学習時には、補助分類器からの損失も最終的な損失に加算しました（重み0.3）。推論時には補助分類器は使用しません。

*   **学習結果と課題:**
    *   CIFAR-10用に調整したGoogLeNet風モデルでも、適切な学習を行えば良好な精度が期待できます。補助分類器や1x1ボトルネックの効果を検証するには、それらの有無で比較実験を行うと良いでしょう。
    *   元のGoogLeNetは非常に深い（22重み層）ですが、CIFAR-10ではそこまでの深さは必ずしも必要ないか、過学習のリスクを高める可能性があります。モデルのサイズとデータセットのサイズのバランスが重要です。
    *   ハイパーパラメータ（学習率、Optimizer、正則化の強さ、補助分類器の損失の重みなど）のチューニングは、性能を最大限に引き出すために依然として重要です。

GoogLeNet (Inceptionアーキテクチャ) は、CNNの設計において「どのように計算資源を効率的に使い、性能を向上させるか」という問題に対する一つの強力な回答を示しました。その後の多くのネットワークアーキテクチャ（Inception v2, v3, v4, Inception-ResNetなど）は、このInceptionのアイデアをさらに発展させたものです。
